In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import Dataset
import os
import transformers

import pandas as pd

from dataset.besstie import dataset_besstie

root_folder = "dataset/besstie/"
splits = {'train': 'train.csv', 'validation': 'valid.csv'}
if not os.path.exists(root_folder):
    os.makedirs(root_folder)
if not os.path.exists(os.path.join(root_folder, splits["train"])) or not os.path.exists(os.path.join(root_folder, splits["validation"])):
    print("Downloading BESSTIE dataset...")
    # Login using e.g. `huggingface-cli login` to access this dataset
    df = pd.read_csv("hf://datasets/unswnlporg/BESSTIE/" + splits["train"])
    df.to_csv(os.path.join(root_folder, splits["train"]), index=False)
    df = pd.read_csv("hf://datasets/unswnlporg/BESSTIE/" + splits["validation"])
    df.to_csv(os.path.join(root_folder, splits["validation"]), index=False)


In [2]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    import random
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    

In [ ]:
#TODO: add ability to choose validation subcategory of the dataset
dataset_CFG = {
    'dataset_name': 'BESSTIE',
    'task': 'Sentiment',
    'variety': 'en-UK',
    'source': 'Google',
}
CFG = {
    'lr': 2e-5,
    'epochs': 8,
    'batch_size': 8,
    'max_length': 200,
    'min_length': 1,
    **dataset_CFG,
    'model_name': 'bert-base-uncased',
    'classification_head': 'conv', # 'linear' or 'conv' or 'lstm'
    'seed': 0,
}

df_train = pd.read_csv(os.path.join(root_folder, splits['train']))
labels_count = df_train["label"].value_counts().sort_index()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(labels_count)
print("Using device:", device)
set_seed(CFG['seed'])

label
0    12092
1     5668
Name: count, dtype: int64
Using device: cuda


In [ ]:
class MultiKernelConvHead(torch.nn.Module):
    def __init__(
        self,
        input_size: int,
        hidden_size: int,
        num_labels: int,
        kernel_sizes=(2, 3, 5),
        dropout=0.1
    ):
        super().__init__()

        self.convs = torch.nn.ModuleList([
            torch.nn.Conv1d(
                in_channels=input_size,
                out_channels=hidden_size,
                kernel_size=k,
                padding=k // 2
            )
            for k in kernel_sizes
        ])

        self.activation = torch.nn.ReLU()
        self.pool = torch.nn.AdaptiveAvgPool1d(1)
        self.dropout = torch.nn.Dropout(dropout)

        self.classifier = torch.nn.Linear(
            hidden_size * len(kernel_sizes),
            num_labels
        )

    def forward(self, x):
        # x: (B, H, L)
        conv_outputs = []

        for conv in self.convs:
            h = self.activation(conv(x))      # (B, C, L)
            h = self.pool(h).squeeze(-1)       # (B, C)
            conv_outputs.append(h)

        x = torch.cat(conv_outputs, dim=1)    # (B, C * num_kernels)
        x = self.dropout(x)
        logits = self.classifier(x)

        return logits

def get_tokenizer_and_model(model_name:str):
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
    model = transformers.AutoModel.from_pretrained(model_name)
    return tokenizer, model

def get_classification_head(method: str, input_size:int, hidden_size: int, num_labels: int):
    if method == "linear":
        return torch.nn.Linear(input_size, num_labels)
    elif method == "conv":
        return torch.nn.Sequential(
            torch.nn.Conv1d(
                in_channels=input_size,
                out_channels=hidden_size,
                kernel_size=3,
                padding=1
            ),
            torch.nn.ReLU(),
            torch.nn.AdaptiveAvgPool1d(1),
            torch.nn.Flatten(),
            torch.nn.Linear(hidden_size, num_labels)
        )
    elif method == "lstm":
        return torch.nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=1,
            batch_first=True,
            bidirectional=True
        )
    elif method == "multi_conv":
        return MultiKernelConvHead(
            input_size=input_size,
            hidden_size=hidden_size,
            num_labels=num_labels,
            kernel_sizes=(2, 3, 5),
            num_channels=128,
            dropout=0.1
        )
    else:
        raise ValueError(f"Unknown classification head method: {method}")


class MyClassifier(torch.nn.Module):
    def __init__(self, base_model_name, classification_head_name, num_labels):
        super().__init__()

        self.tokenizer, self.base_model = get_tokenizer_and_model(base_model_name)
        self.hidden_size = self.base_model.config.hidden_size
        self.dropout = torch.nn.Dropout(self.base_model.config.hidden_dropout_prob)

        self.classification_head_name = classification_head_name

        self.classification_head = get_classification_head(
            classification_head_name, self.hidden_size, num_labels
        )

        if classification_head_name == "lstm":
            self.output_layer = torch.nn.Linear(self.hidden_size//2, num_labels)
    
    def get_tokenizer(self) -> transformers.PreTrainedTokenizer:
        return self.tokenizer
    
    def forward(self, inputs):
        outputs = self.base_model(**inputs)
        sequence = self.dropout(outputs.last_hidden_state)

        if self.classification_head_name == "linear":
            cls_rep = sequence[:, 0, :]
            logits = self.classification_head(cls_rep)

        elif self.classification_head_name == "conv":
            x = sequence.transpose(1, 2)
            logits = self.classification_head(x)

        elif self.classification_head_name == "lstm":
            lstm_out, _ = self.classification_head(sequence)
            cls_rep = lstm_out[:, 0, :]
            logits = self.output_layer(cls_rep)

        return logits


In [44]:
def validate(model, val_loader, criterion, device):
    model.eval()
    val_acc = 0.0
    val_loss = 0.0
    for batch in val_loader:
        inputs = {
            'input_ids': batch['input_ids'].to(device),
            'attention_mask': batch['attention_mask'].to(device)
        }
        local_labels = batch['label'].to(device)
        outputs = model(inputs)
        loss = criterion(outputs, local_labels)
        _, preds = torch.max(outputs, dim=1)
        val_acc += torch.sum(preds == local_labels).item()
        val_loss += loss.item()

    return val_loss / len(val_loader), val_acc / len(val_loader.dataset)

In [45]:
import wandb
run_name = "Test get classification head conv"
# run_name = None

run = wandb.init(
    entity="elena-nespolo02-politecnico-di-torino",
    project="Figurative Analysis",
    name=run_name,
    config=CFG,
    tags=[CFG['dataset_name'], CFG['task'], CFG['model_name']]
)


In [ ]:
import tqdm


model_name = CFG['model_name']

tokenizer, model = get_tokenizer_and_model(model_name)

tokenizer = transformers.BertTokenizer.from_pretrained(model_name)

# load classifier model
# model = transformers.BertForSequenceClassification.from_pretrained(
#     model_name,
#     num_labels=2
# ).to(device)

model = MyClassifier(
    base_model_name=model_name,
    classification_head_name=CFG['classification_head'],
    num_labels=2
).to(device)

train_ds = dataset_besstie.BesstieDataSet(
    root_folder=root_folder,
    file_name=splits['train'],
    classes=['0', '1'],
    tokenizer=tokenizer,
    min_length=CFG['min_length'],
    max_length=CFG['max_length'],
    variety=CFG['variety'],
    source=CFG['source'],
    task=CFG['task']
)

val_ds = dataset_besstie.BesstieDataSet(
    root_folder=root_folder,
    file_name=splits['validation'],
    classes=['0', '1'],
    tokenizer=tokenizer,
    min_length=CFG['min_length'],
    max_length=CFG['max_length'],
    variety=CFG['variety'],
    source=CFG['source'],
    task=CFG['task']
)

optimizer = torch.optim.Adam(model.parameters(), lr=CFG['lr'])
criterion = torch.nn.CrossEntropyLoss(
    weight=torch.tensor(labels_count.values/sum(labels_count), dtype=torch.float).to(device)
)

train_loader = torch.utils.data.DataLoader(
    train_ds,
    batch_size=CFG['batch_size'],
    shuffle=True
)

val_loader = torch.utils.data.DataLoader(
    val_ds,
    batch_size=CFG['batch_size'],
    shuffle=False
)

#TODO: gradient accumulation to reduce memory usage?
# accumulation_steps = 4  # Effective batch size = batch_size * accumulation_steps
# for i, batch in enumerate(train_dataloader):
#     outputs = model(**batch)
#     loss = outputs.loss / accumulation_steps
#     loss.backward()
#     if (i + 1) % accumulation_steps == 0:
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
#         optimizer.step()
#         scheduler.step()
#         model.zero_grad()

for epoch in range(CFG['epochs']):
    model.train()

    train_loss = 0.0
    print(f"Epoch {epoch+1}/{CFG['epochs']}")
    
    pbar = tqdm.tqdm(train_loader)
    for batch in pbar:
        inputs = {
            'input_ids': batch['input_ids'].to(device),
            'attention_mask': batch['attention_mask'].to(device)
        }
        local_labels = batch['label'].to(device)
        outputs = model(inputs)
        loss = criterion(outputs, local_labels)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        train_loss += loss.item()

    val_loss, val_acc = validate(model, val_loader, criterion, device)

    epoch_loss = train_loss / len(train_loader)
    run.log({"train_loss": epoch_loss, "val_loss": val_loss, "val_acc": val_acc})
    print(f"Training Loss: {epoch_loss:.4f}")

run.finish()


MyClassifier(
  (base_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

100%|██████████| 228/228 [18:32<00:00,  4.88s/it]


Training Loss: 0.2450
Epoch 2/8


100%|██████████| 228/228 [18:50<00:00,  4.96s/it]


Training Loss: 0.0760
Epoch 3/8


  2%|▏         | 5/228 [00:29<22:16,  5.99s/it]


KeyboardInterrupt: 

In [49]:
run.finish()

train_loss,█▁
val_acc,▁█
val_loss,▁█
train_loss,0.076
val_acc,0.95968
val_loss,0.23824
